In [1]:
import tkinter as tk
from tkinter import ttk
from PIL import Image, ImageOps
from pillow_heif import register_heif_opener
from docx import Document
from docx.shared import Inches
from docx.shared import Pt
from docx.oxml import OxmlElement
from docx.oxml.ns import qn
from lxml import etree
from docx.enum.table import WD_TABLE_ALIGNMENT
import traceback
from docx.enum.text import WD_ALIGN_PARAGRAPH

In [5]:
df = pd.read_csv('PRG Sales Comparables Table.csv', encoding='latin1')

In [2]:
from docx import Document
from docx.oxml.ns import qn
from docx.oxml import OxmlElement
from docx.shared import Inches, Pt
from docx.oxml import OxmlElement

def set_cell_borders(cell, border_color="000000", border_width=4):
    # Create an XML element for cell properties if it doesn't exist
    tc_pr = cell._element.xpath('.//w:tcPr')
    if not tc_pr:
        tc_pr = OxmlElement('w:tcPr')
        cell._element.insert(0, tc_pr)
    else:
        tc_pr = tc_pr[0]
    
    # Create a new border element
    borders = OxmlElement('w:tcBorders')
    border_attrs = {
        'w:top': border_color,
        'w:left': border_color,
        'w:bottom': border_color,
        'w:right': border_color,
        'w:insideH': border_color,
        'w:insideV': border_color
    }
    
    for border in border_attrs:
        border_elem = OxmlElement(border)
        border_elem.set(qn('w:val'), 'single')
        border_elem.set(qn('w:sz'), str(border_width))  # Border width (1/8 pt)
        border_elem.set(qn('w:space'), '0')
        borders.append(border_elem)

    # Append the borders to the cell properties
    tc_pr.append(borders)

def set_cell_fill(cell, color):
    # Define shading for cell fill color
    shading = OxmlElement('w:shd')
    shading.set(qn('w:fill'), color)
    cell._element.get_or_add_tcPr().append(shading)

def set_paragraph_spacing(cell, spacing_before=0, spacing_after=0):
    # Set paragraph spacing to single and specific before/after spacing
    for para in cell.paragraphs:
        para_format = para.paragraph_format
        para_format.space_after = Pt(spacing_after)
        para_format.space_before = Pt(spacing_before)
        para_format.line_spacing = Pt(12)  # Single line spacing (12 pt)

def clear_cell_text(cell):
    # Clear all text from the cell
    cell.text = ''
    set_paragraph_spacing(cell)  # Ensure paragraph spacing is set

from docx.shared import Pt
from docx.enum.text import WD_ALIGN_PARAGRAPH

def set_cell_text(cell, text, font_name='Times New Roman', font_size=11, bold=True, alignment='left'):
    
    # Set text in the cell
    cell.text = text
    
    # Determine alignment
    if alignment == 'left':
        align = WD_ALIGN_PARAGRAPH.LEFT
    elif alignment == 'center':
        align = WD_ALIGN_PARAGRAPH.CENTER
    elif alignment == 'right':
        align = WD_ALIGN_PARAGRAPH.RIGHT
    else:
        raise ValueError("Invalid alignment value. Choose from 'left', 'center', or 'right'.")

    # Apply formatting to paragraphs
    for para in cell.paragraphs:
        para_format = para.paragraph_format
        para_format.alignment = align  # Set alignment
        para_format.space_after = Pt(0)  # Remove space after the paragraph
        
        for run in para.runs:
            run.font.name = font_name
            run.font.size = Pt(font_size)
            run.font.bold = bold
        
        # Apply font name to paragraph style
        para.style.font.name = font_name

def fill_row_with_cells(table, row_idx, fill_color="FFFFFF"):
    for col_idx in range(2):  # Two columns
        cell = table.cell(row_idx, col_idx)
        set_cell_borders(cell, border_color="000000", border_width=4)  # black border with width 4
        set_cell_fill(cell, color=fill_color)  # white fill or other
        clear_cell_text(cell)  # Clear text and set paragraph spacing

def create_table_with_content(doc):
    from docx.oxml import OxmlElement
    from docx.oxml.ns import qn
    from docx.enum.text import WD_ALIGN_PARAGRAPH

    # Set document margins
    section = doc.sections[0]
    section.left_margin = Inches(1)
    section.right_margin = Inches(1)

    # Add a table with 35 rows and 2 columns
    table = doc.add_table(rows=35, cols=2)

    # Content for merged and normal cells
    row_data = [
        (0, "D0D0D0"),  # Gray color (White, background 1, darker 25%)
        (1, "FFFFFF"),
        (2, "FFFFFF"),
        (3, "D0D0D0"),
    ]

    # Populate table with initial data
    for row_idx, fill_color in row_data:
        if row_idx == 0:  # Merged cell
            cell_1 = table.cell(row_idx, 0)
            cell_2 = table.cell(row_idx, 1)
            cell_1.merge(cell_2)
            set_cell_borders(cell_1, border_color="000000", border_width=4)
            set_cell_fill(cell_1, color=fill_color)
            # Set the specific text for the first gray row with correct spacing
            set_cell_text(cell_1, "SALE COMPARABLE\nPRG Reference: ", font_name='Times New Roman', font_size=11, bold=True, alignment = 'left')
        elif row_idx == 1:  # Special case for second row
            cell_1 = table.cell(row_idx, 0)
            cell_2 = table.cell(row_idx, 1)
            set_cell_borders(cell_1, border_color="000000", border_width=4)
            set_cell_borders(cell_2, border_color="000000", border_width=4)
            set_cell_fill(cell_1, color="FFFFFF")
            set_cell_fill(cell_2, color="FFFFFF")
        elif row_idx == 2:  # Special case for second row
            cell_1 = table.cell(row_idx, 0)
            cell_2 = table.cell(row_idx, 1)
            set_cell_borders(cell_1, border_color="000000", border_width=4)
            set_cell_borders(cell_2, border_color="000000", border_width=4)
            set_cell_fill(cell_1, color="FFFFFF")
            set_cell_fill(cell_2, color="FFFFFF")
            # Set text for the second row
            set_cell_text(cell_1, "SALE PHOTO", font_name='Times New Roman', font_size=10, bold=True, alignment='center')
            set_cell_text(cell_2, "TAX MAP", font_name='Times New Roman', font_size=10, bold=True, alignment='center')
        else:  # Non-merged cells
            fill_row_with_cells(table, row_idx, fill_color)
    
    # Add additional rows with alternating gray and white merged cells
    for i in range(4, 9):
        fill_row_with_cells(table, i, "FFFFFF")
    
    add_merged_row(table, 9, "D0D0D0")
    
    for i in range(10, 22):
        fill_row_with_cells(table, i, "FFFFFF")
    
    add_merged_row(table, 22, "D0D0D0")
    
    for i in range(23, 30):
        fill_row_with_cells(table, i, "FFFFFF")
    
    add_merged_row(table, 30, "D0D0D0")
    
    add_merged_row(table, 31, "FFFFFF")
    
    # Save the document
    doc.save('extended_table_with_no_text.docx')

def add_merged_row(table, row_idx, fill_color="D0D0D0"):
    cell_1 = table.cell(row_idx, 0)
    cell_2 = table.cell(row_idx, 1)
    cell_1.merge(cell_2)
    set_cell_borders(cell_1, border_color="000000", border_width=4)
    set_cell_fill(cell_1, color=fill_color)
    clear_cell_text(cell_1)  # Clear text and set paragraph spacing

# Create a new Document
doc = Document()
create_table_with_content(doc)
